<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](https://github.com/jwross24/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/mountain/art1131.jpg?raw=1)|![](https://github.com/jwross24/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/forest/cdmc317.jpg?raw=1)|

The problem is realively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several differnet possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D()
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = res.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(res.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/mountain/*`) and images of forests (`./data/forest/*`). Treat mountains as the postive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pretrained layers from resnet
4. Report your model's accuracy

In [1]:
%tensorflow_version 2

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `2`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [2]:
!pip install tf-nightly-gpu

  Using cached https://files.pythonhosted.org/packages/d6/84/759f5dd23fec8ba71952d97bcc7e2c9d7d63bdc582421f3cd4be845f0c98/gast-0.3.3-py2.py3-none-any.whl
ERROR: tensorflow-federated 0.11.0 requires enum34~=1.1, which is not installed.
ERROR: tensorflow 2.1.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.
ERROR: tensorflow-federated 0.11.0 has requirement attrs~=18.2, but you'll have attrs 19.3.0 which is incompatible.
ERROR: tensorflow-federated 0.11.0 has requirement cachetools~=3.1.1, but you'll have cachetools 4.0.0 which is incompatible.
ERROR: tensorflow-federated 0.11.0 has requirement grpcio~=1.24.3, but you'll have grpcio 1.27.2 which is incompatible.
ERROR: tensorflow-federated 0.11.0 has requirement tensorflow~=2.0.0, but you'll have tensorflow 2.1.0 which is incompatible.
ERROR: tensorflow-federated 0.11.0 has requirement tensorflow-addons~=0.6.0, but you'll have tensorflow-addons 0.8.2 which is incompatible.
ERROR: tensor2tensor 1.14.1 has re

In [3]:
try:
    import tensorflow.compat.v2 as tf
except Exception:
    pass
tf.enable_v2_behavior()
print(tf.__version__)

2.1.0


In [4]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Load in Data

![skimage-logo](https://scikit-image.org/_static/img/logo.png)

Check out out [`skimage`](https://scikit-image.org/) for useful functions related to processing the images. In particular checkout the documentation for `skimage.io.imread_collection` and `skimage.transform.resize`.

In [0]:
from skimage.io import imread_collection
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import numpy as np

# Your path
forests = './data/forest/*.jpg'
mountains = './data/mountain/*.jpg'

# Create a collection with the available images
forests = imread_collection(forests).concatenate()
mountains = imread_collection(mountains).concatenate()

y_0 = np.zeros(forests.shape[0])
y_1 = np.ones(mountains.shape[0])

X = np.concatenate([forests, mountains])
X = resize(X, (702, 224, 224, 3))
y = np.concatenate([y_0, y_1])

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [66]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((561, 224, 224, 3), (141, 224, 224, 3), (561,), (141,))

## Instantiate Model

In [0]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model  # This is the functional API

resnet = ResNet50(input_shape=(224, 224, 3),
                  weights='imagenet',
                  include_top=False)

In [0]:
# Freeze the layers we don't want to train
# Here, we choose all of them
for layer in resnet.layers:
    layer.trainable = False

In [0]:
x = resnet.output
x = GlobalAveragePooling2D()(x)  # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [0]:
from tensorflow.keras.optimizers import SGD
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Fit Model

In [71]:
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Train on 561 samples, validate on 141 samples
Epoch 1/10
561/561 [==============================] - 5s 9ms/sample - loss: 0.3274 - accuracy: 0.8770 - val_loss: 0.7529 - val_accuracy: 0.4043
Epoch 2/10
561/561 [==============================] - 2s 3ms/sample - loss: 0.0717 - accuracy: 0.9733 - val_loss: 0.6669 - val_accuracy: 0.5887
Epoch 3/10
561/561 [==============================] - 2s 3ms/sample - loss: 0.0160 - accuracy: 0.9947 - val_loss: 0.7627 - val_accuracy: 0.5887
Epoch 4/10
561/561 [==============================] - 2s 3ms/sample - loss: 0.0120 - accuracy: 0.9947 - val_loss: 0.7033 - val_accuracy: 0.5887
Epoch 5/10
561/561 [==============================] - 2s 3ms/sample - loss: 0.0124 - accuracy: 0.9964 - val_loss: 0.7692 - val_accuracy: 0.4113
Epoch 6/10
561/561 [==============================] - 2s 3ms/sample - loss: 0.1872 - accuracy: 0.9412 - val_loss: 1.3724 - val_accuracy: 0.5887
Epoch 7/10
561/561 [==============================] - 2s 3ms/sample - loss: 0.0581 - accur

Train on 561 samples, validate on 141 samples
Epoch 1/5
561/561 [==============================] - 39s 69ms/sample - loss: 0.1216 - accuracy: 0.9715 - val_loss: 1.4221 - val_accuracy: 0.0000e+00
Epoch 2/5
561/561 [==============================] - 35s 62ms/sample - loss: 0.0410 - accuracy: 0.9875 - val_loss: 2.9222 - val_accuracy: 0.0000e+00
Epoch 3/5
561/561 [==============================] - 35s 62ms/sample - loss: 0.0646 - accuracy: 0.9697 - val_loss: 0.9962 - val_accuracy: 0.0000e+00
Epoch 4/5
561/561 [==============================] - 39s 70ms/sample - loss: 0.0723 - accuracy: 0.9733 - val_loss: 0.2627 - val_accuracy: 1.0000
Epoch 5/5
561/561 [==============================] - 38s 68ms/sample - loss: 0.0333 - accuracy: 0.9893 - val_loss: 0.4859 - val_accuracy: 1.0000


# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [73]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential

model = Sequential([
    Conv2D(32, (10,10), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D((5,5)),
    Conv2D(64, (5,5), activation='relu'),
    MaxPooling2D((3,3)),
    Conv2D(64, (5,5), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 215, 215, 32)      9632      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 43, 43, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 39, 39, 64)        51264     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          102464    
_________________________________________________________________
flatten (Flatten)            (None, 5184)              0         
_________________________________________________________________
dense_40 (Dense)             (None, 64)                3

In [74]:
# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Fit model
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Train on 561 samples, validate on 141 samples
Epoch 1/10
561/561 [==============================] - 1s 3ms/sample - loss: 0.5705 - accuracy: 0.7184 - val_loss: 0.4050 - val_accuracy: 0.8369
Epoch 2/10
561/561 [==============================] - 1s 1ms/sample - loss: 0.3375 - accuracy: 0.8538 - val_loss: 0.3444 - val_accuracy: 0.8582
Epoch 3/10
561/561 [==============================] - 1s 1ms/sample - loss: 0.3159 - accuracy: 0.8734 - val_loss: 0.4331 - val_accuracy: 0.8369
Epoch 4/10
561/561 [==============================] - 1s 1ms/sample - loss: 0.2789 - accuracy: 0.8895 - val_loss: 0.2728 - val_accuracy: 0.9007
Epoch 5/10
561/561 [==============================] - 1s 1ms/sample - loss: 0.2707 - accuracy: 0.8788 - val_loss: 0.3761 - val_accuracy: 0.8440
Epoch 6/10
561/561 [==============================] - 1s 1ms/sample - loss: 0.2463 - accuracy: 0.9180 - val_loss: 0.2920 - val_accuracy: 0.9007
Epoch 7/10
561/561 [==============================] - 1s 1ms/sample - loss: 0.2055 - accur

# Custom CNN Model with Image Manipulations
## *This a stretch goal, and it's relatively difficult*

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language